In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

/usr/local/lib/python3.5/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


# Here, we develop tools to pick up data needed for calibration

In [2]:
from LeCheapEyeTracker import Client, Server, Stimulation

import numpy as np
import matplotlib.pyplot as plt
import time

import cv2
from vispy import gloo, app

## First, we need to pick up stimuli x_positions.

In [3]:
class NeoStimulation(Stimulation):
    def get_stimulus(self, t0, t):
        i = int((t-t0)//self.transition_lag)
        stim_x, stim_y = self.tabPos[i]
        return self.draw_stimulus((stim_x, stim_y)), stim_x

In [4]:
class NeoClient(Client):
    def __init__(self, et, timeline, stim_type = 'calibration'):
        Client.__init__(self, et, timeline, stim_type)
        self.stimulation = NeoStimulation(self.width, self.height, stim_type = stim_type)
        self.stims_posX = np.array([])
        self.stims_time = np.array([])
        
    def on_draw(self, event):
        gloo.clear('black')
        if time.time()-self.start < self.timeline.max():
            image, x = self.stimulation.get_stimulus(t0 = self.start, t = time.time())
            self.stims_posX = np.append(self.stims_posX, x)
            self.stims_time = np.append(self.stims_time, time.time()-self.start)
            self.program['texture'][...] = image.astype(np.uint8).reshape((self.height, self.width, 3))
            self.program.draw('triangle_strip')
        else:
            self.close()

In [5]:
fps = 100
screen = NeoClient(et=Server(), timeline=np.linspace(0, 7., 3.*fps))
screen.app.run()

window size :  (2560, 1440)
could not grab a frame / detect the eyes position
could not grab a frame / detect the eyes position
could not grab a frame / detect the eyes position
could not grab a frame / detect the eyes position
could not grab a frame / detect the eyes position
could not grab a frame / detect the eyes position
could not grab a frame / detect the eyes position
could not grab a frame / detect the eyes position
could not grab a frame / detect the eyes position
could not grab a frame / detect the eyes position
could not grab a frame / detect the eyes position
could not grab a frame / detect the eyes position
could not grab a frame / detect the eyes position
could not grab a frame / detect the eyes position
could not grab a frame / detect the eyes position
could not grab a frame / detect the eyes position
could not grab a frame / detect the eyes position
could not grab a frame / detect the eyes position
could not grab a frame / detect the eyes position
could not grab a frame

/usr/local/lib/python3.5/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)


In [6]:
print (screen.stims_posX)

[ 0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5
  0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5
  0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5
  0.5   0.33  0.33  0.33  0.33  0.33  0.33  0.33  0.33  0.33  0.33  0.33
  0.33  0.33  0.33  0.33  0.33  0.33  0.33  0.33  0.33  0.5   0.5   0.5
  0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5
  0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5
  0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5
  0.5   0.67  0.67  0.67  0.67  0.67  0.67  0.67  0.67  0.67  0.67  0.67
  0.67  0.67  0.67  0.67  0.67  0.67  0.67  0.67  0.5   0.5   0.5   0.5
  0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5
  0.5   0.5   0.5   0.5 ]
